<a href="https://colab.research.google.com/github/BartoszTajak/https-czytamyetykiety.pl/blob/master/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from bs4 import BeautifulSoup as bs
import requests
import time
import numpy as np
import pandas as pd
import os
from google.colab import drive
import matplotlib.pyplot as plt
import re
import math
drive.mount('/content/drive')

In [ ]:
# Link to current website
base_link = r'https://czytamyetykiety.pl/producenci-zywnosci/'
r = requests.get(base_link)
soup = bs(r.content, "html.parser")
#list of all tabs (A,B,C...)
all_tabs = soup.find_all('ul', {'class': 'css-pub6cd e1vaxell7'})[0].select('li')
all_tabs =['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','Q','P','R','S','T','U','V','W','X','Y','Z']
    


# page by page  (A,B,C....)
for lettter in all_tabs:
    link = f'https://czytamyetykiety.pl/producenci-zywnosci/?letter={lettter}'
    r = requests.get(link)
    soup = bs(r.content, "html.parser")
    companies = soup.find_all('ul', {'class': 'css-1xoq838 e1vaxell1'})[0].select('li')
    # create empty DataFrame
    df = pd.DataFrame()
    # all brands on page 
    for i in companies:
        try:
            data_to_frame=[]
            print('Następny producent : ',i.get_text())
            # add to producer's list
            Producent = i.get_text()

            href = i.find('a').attrs['href']
            company_link='https://czytamyetykiety.pl'+href
            time.sleep(1)


            r = requests.get(company_link)
            soup = bs(r.content, "html.parser")
            all_products = soup.find_all('div', {'class': 'row css-xqnzc8 e1jjiacg7'})[0]
            lenght = len(all_products.find_all('a' , {'class' : 'css-1ri2k5u ejmlxjt3'}))
            # all items on page 
            for y in range(lenght):
                main_list = []
                try:
                    main_list.append(Producent)
                    every_item = all_products.find_all('a' , {'class' : 'css-1ri2k5u ejmlxjt3'})[y].attrs['href']

                    # link to each iteam 
                    link_to_product = 'https://czytamyetykiety.pl'+every_item
                    r = requests.get(link_to_product)
                    soup = bs(r.content, "html.parser")
                    every_item = soup.find_all('ul', {'class': 'css-1dwqukr eq13dfd10'})[0].select('li')


                    name_of_product = soup.find('h1' , {'class' : 'css-3ro1ko eq13dfd5'})
                    Produkt = name_of_product.get_text()
                    main_list.append(Produkt)
                    lista_skaldnikow = []
                    for x in every_item:
                        skladnik = x.get_text()
                        lista_skaldnikow.append(skladnik)

                    every_item = soup.find_all('div', {'class': 'css-12npjva e1sc61rg3'})[0]
                    if every_item.find('div' , {'class' : 'css-6sex43'}) != None:
                        ocena = 'Produkt sredni'
                    elif every_item.find('div' , {'class' : 'css-jtr8y8'}) != None:
                        ocena = 'Produkt dobry'
                    else:
                        ocena = 'Produkt zly'

                    main_list.append(lista_skaldnikow)
                    main_list.append(ocena)
                    rr = requests.get(link_to_product)
                    soup = bs(rr.content, "html.parser")
                    bialko_weg_fett = []
                    lenght = len(soup.find_all('ul', {'class': 'css-1w0253d eq13dfd8'})[0].find_all('div', {
                        'class': 'row css-166ixqk els7ok21'}))
                    for i in range(1, lenght - 1):
                        name = soup.find_all('ul', {'class': 'css-1w0253d eq13dfd8'})[0].find_all('div', {
                            'class': 'row css-166ixqk els7ok21'})[i].select('div')[0].get_text()
                        value = soup.find_all('ul', {'class': 'css-1w0253d eq13dfd8'})[0].find_all('div', {
                            'class': 'row css-166ixqk els7ok21'})[i].select('div')[1].get_text()
                        bialko_weg_fett.append(name)
                        bialko_weg_fett.append(value)

                    main_list.append(bialko_weg_fett)

                    # Adding extra inf ( slodyczne , napoje etc...)
                    all_tabs = soup.find_all('ol', {'class': 'css-1ifqhry eir1p9f2'})[0].select('li')
                    len(all_tabs)
                    artykow = len(all_tabs) - 2
                    all_tabs = soup.find_all('ol', {'class': 'css-1ifqhry eir1p9f2'})[0].select('li')[artykow]
                    all_tabs.get_text()
                    main_list.append(all_tabs.get_text())

                    # Adding extra inf ( produkt weganski etc...)
                    all_tabs = soup.find_all('div', {'class': 'row css-be81vk e1sc61rg2'})[0].find_all('span',{'data-tip': True})
                    extra_lista=[]
                    for i in all_tabs:
                        extra_lista.append(i['data-tip'])

                    main_list.append(extra_lista)

                except:
                    print('error')


                print(main_list)
                data_to_frame.append(main_list)
                main_list=[]



            for i in data_to_frame:
                if len(i) == 7 :
                    df2 = pd.DataFrame([i], columns=['Producent', 'Produkt', 'Sklad','Jakosc produktu', 'Wartosc odrz.','Kategoria', 'Extra inf.'])
                    df = df.concat(df2)
                    df.to_csv(f'{Producent}.csv', sep=';', index=False, encoding='utf-8-sig')
                else:
                    print('error niezgodna ilosc skladników')

        except:
            print('some error')


    link = f'https://czytamyetykiety.pl/producenci-zywnosci/?letter={lettter}'
    r = requests.get(link)
    soup = bs(r.content, "html.parser")
    companies = soup.find_all('ul', {'class': 'css-1xoq838 e1vaxell1'})[0].select('li')
    if len(companies)==51:

        link = f'https://czytamyetykiety.pl/producenci-zywnosci/page/2/?letter={lettter}'
        r = requests.get(link)
        soup = bs(r.content, "html.parser")
        companies = soup.find_all('ul', {'class': 'css-1xoq838 e1vaxell1'})[0].select('li')
        for i in companies:
            try:
                data_to_frame = []
                print('Następny producent : ', i.get_text())
                # add to producer's list
                Producent = i.get_text()

                href = i.find('a').attrs['href']
                company_link = 'https://czytamyetykiety.pl' + href
                time.sleep(1)


                r = requests.get(company_link)
                soup = bs(r.content, "html.parser")
                all_products = soup.find_all('div', {'class': 'row css-xqnzc8 e1jjiacg7'})[0]
                lenght = len(all_products.find_all('a', {'class': 'css-1ri2k5u ejmlxjt3'}))
                # all products on page
                for y in range(lenght):
                    main_list = []
                    try:
                        main_list.append(Producent)
                        every_item = all_products.find_all('a', {'class': 'css-1ri2k5u ejmlxjt3'})[y].attrs['href']

                        # link to every item on page
                        link_to_product = 'https://czytamyetykiety.pl' + every_item
                        r = requests.get(link_to_product)
                        soup = bs(r.content, "html.parser")
                        every_item = soup.find_all('ul', {'class': 'css-1dwqukr eq13dfd10'})[0].select('li')

                        name_of_product = soup.find('h1', {'class': 'css-3ro1ko eq13dfd5'})
                        Produkt = name_of_product.get_text()
                        main_list.append(Produkt)
                        lista_skaldnikow = []
                        for x in every_item:

                            skladnik = x.get_text()
                            lista_skaldnikow.append(skladnik)


                        every_item = soup.find_all('div', {'class': 'css-12npjva e1sc61rg3'})[0]
                        if every_item.find('div', {'class': 'css-6sex43'}) != None:
                            ocena = 'Produkt sredni'
                        elif every_item.find('div', {'class': 'css-jtr8y8'}) != None:
                            ocena = 'Produkt dobry'
                        else:
                            ocena = 'Produkt zly'

                        main_list.append(lista_skaldnikow)
                        main_list.append(ocena)
                        rr = requests.get(link_to_product)
                        soup = bs(rr.content, "html.parser")


                        bialko_weg_fett=[]
                        lenght = len(soup.find_all('ul', {'class': 'css-1w0253d eq13dfd8'})[0].find_all('div', {
                            'class': 'row css-166ixqk els7ok21'}))
                        for i in range(1, lenght - 1):
                            name = soup.find_all('ul', {'class': 'css-1w0253d eq13dfd8'})[0].find_all('div', {
                                'class': 'row css-166ixqk els7ok21'})[i].select('div')[0].get_text()
                            value = soup.find_all('ul', {'class': 'css-1w0253d eq13dfd8'})[0].find_all('div', {
                                'class': 'row css-166ixqk els7ok21'})[i].select('div')[1].get_text()
                            bialko_weg_fett.append(name)
                            bialko_weg_fett.append(value)


                        main_list.append(bialko_weg_fett)

                        # Dodawanie kategori ( slodyczne , napoje etc...)
                        all_tabs = soup.find_all('ol', {'class': 'css-1ifqhry eir1p9f2'})[0].select('li')
                        len(all_tabs)
                        artykow = len(all_tabs) - 2
                        all_tabs = soup.find_all('ol', {'class': 'css-1ifqhry eir1p9f2'})[0].select('li')[artykow]
                        all_tabs.get_text()
                        main_list.append(all_tabs.get_text())

                        # Dodawanie dodatkowych inf ( produkt weganski etc...)
                        all_tabs = soup.find_all('div', {'class': 'row css-be81vk e1sc61rg2'})[0].find_all('span', {
                            'data-tip': True})
                        extra_lista = []
                        for i in all_tabs:
                            extra_lista.append(i['data-tip'])

                        main_list.append(extra_lista)

                    except:
                        print('error')


                    print(main_list)
                    data_to_frame.append(main_list)
                    main_list = []

                for i in data_to_frame:
                    if len(i) == 7:
                        df2 = pd.DataFrame([i],
                                           columns=['Producent', 'Produkt', 'Sklad', 'Jakosc produktu', 'Wartosc odrz.',
                                                    'Kategoria', 'Extra inf.'])
                        df = df.append(df2)
                        df.to_csv(f'{Producent}.csv', sep=';', index=False, encoding='utf-8-sig')
                    else:
                        print('error niezgodna ilosc skladników')

            except:
                print('some error')

    else:
        pass





In [ ]:
# create empty DataFrame
all_product = pd.DataFrame()
mypath = r'/content/drive/MyDrive/Python/czytamy_etykiety/csv'
csv_list = os.listdir(mypath)
# combain all csv files
for i in csv_list:   
    df = pd.read_csv(mypath+'//'+i, sep =';',encoding='utf-8')
    all_product = pd.concat([all_product,df])


In [ ]:
# Division according to quality

all_product = all_product.rename(columns={"Jakosc produktu": "quality of product"})
quality=(all_product['quality of product'].value_counts()).to_dict()


plt.figure(figsize=(14,10))
plt.bar(*zip(*quality.items()))
plt.grid()
plt.show()

In [ ]:
# Division according to category

lista = all_product['Kategoria'].value_counts()

plt.figure(figsize=(28,15))
plt.xticks(rotation=90)
plt.grid()
D = lista.to_dict()

D_name = [i[0] for i in list(D.items()) if i[1] > 5 ] 
D_num = [i[1] for i in list(D.items()) if i[1] > 5 ] 
D = dict(zip(D_name, D_num))

plt.bar(*zip(*D.items()))

plt.show()

In [ ]:
# Add new column with number of ingredients
all_product['Nr. of  Ingredient'] = all_product['Sklad'].apply(lambda data: len(data.split(',')))

In [ ]:
# extracting kcal/kj from data 
all_product['Energy'] = all_product['Wartosc odrz.'].str.extract(r'(\d*\s*kcal|\d*\s*Kcal|\d*\s*kj|\d*\s*Kj|\d*\s*kJ)')
all_product['Energy'] = all_product['Energy'].astype(str)


# clead data and change to kj 
def energia(x):
  x = x.replace(' ','')
  if (('kJ' in x )or('KJ' in x)) or ('kj' in x):
    x = re.sub("(kj|KJ|kJ)", "", x)
    return x
  elif ('Kcal' in x) or ('kcal' in x):
    x = re.sub("(kcal|Kcal)", "", x)
    x = int(x)
    return round(4.18*x,2)
  else:
    return x


all_product['Energy'] = all_product['Energy'].apply(energia)


# clead data from '' , 'nan'
all_product.reset_index(inplace=True,drop=True)
all_product.drop(all_product.index[all_product['Energy'] == ''], inplace = True)
all_product.drop(all_product.index[all_product['Energy'] == 'nan'], inplace = True)
all_product['Energy'] = all_product['Energy'].apply(lambda x : round(float(x) /4.18,2))
all_product = all_product.drop(all_product[all_product['Energy'] >900].index)


In [ ]:
def value_extract(x):
  if type(x) == str:
    try:
      x = x.replace(',','.')
      x = x.replace(' ','')
      return round(float(x),2)
    except:
      return 0

# extracting protein,carbohydrates,fat from data 

all_product['protein'] = all_product['Wartosc odrz.'].str.extract(r'(..a.k..{1,5},.{1,5}\d{1,3},*d{0,2})')

all_product['carbohydrates'] = all_product['Wartosc odrz.'].str.extract(r'(glowodany.{1,12}\d{1,3})')

all_product['fat'] = all_product['Wartosc odrz.'].str.extract(r'(uszcz.{1,5},.{1,5}\d{1,3},*d{0,2})')

list_of_columns = ['protein','carbohydrates','fat']

for column in list_of_columns:
  all_product[column] = all_product[column].str.extract(r'(\d{1,3},*\d{0,3})')
  all_product[column] = all_product[column].apply(value_extract)
  all_product[column].fillna(0, inplace = True)
  all_product[column]=all_product[column].astype(np.float16)
  all_product[column]=all_product[column].apply(lambda x: round(x,1))



all_product['Temp'] = all_product['protein'] + all_product['carbohydrates'] + all_product['fat']
all_product = all_product.drop(all_product[all_product['Temp'] >100].index)
all_product.drop(['Temp'], axis=1,inplace=True)

In [ ]:
all_product.reset_index(inplace=True,drop=True)
all_product.to_excel('/content/drive/MyDrive/ALLES.xlsx')
all_product